In [1]:
import numpy as np
import sys, os
cwd_split = os.getcwd().split(os.sep)
root = os.path.abspath("")
sys.path.insert(0, f"{root}/build/python")

import mitsuba as mi
# mi.set_variant("cuda_ad_rgb")
mi.set_variant('cuda_ad_rgb_double', 'llvm_ad_rgb_double')
import drjit as dr

from matplotlib import pyplot as plt

from gaussnewton_test import conjugate_gradient, AlbedoScene

In [2]:
# from mitsuba.scalar_rgb import Transform4f as T

res = 32
scene_init = mi.load_dict({
    'type': 'scene',
    'integrator': {'type': 'prb'},
    'plane': {
                'type': 'rectangle',
                'bsdf': { 'type': 'diffuse',
                            'reflectance': {
                                'type': 'rgb',
                                'value': [0.1, 1.0, 0.1]
                            } 
                         }
    },
    'sensor': {
        'type': 'perspective',
        'fov': 60,
        'to_world': mi.ScalarTransform4f.look_at(
            origin=[0, 0, -2],
            target=[0, 0, 0],
            up=[0, -1, 0]
        ),
        'film': {
            'type': 'hdrfilm',
            'width': res,
            'height': res,
            #'filter': { 'type': 'box' }
        }
    },
    'light': {
        'type': 'constant',
        'radiance': 3.0 
        #'position': [0.0, 5.0, 0.0],
        #'intensity': {
        #'type': 'spectrum',
        #'value': 1.0,
        #}
    }
    
})

scene_dict = {
            'type': 'scene',
            'integrator': {
                'type': 'direct',
                # 'max_depth': 2
            },
            'plane': {
                'type': 'rectangle',
                'bsdf': {
                    'type': 'diffuse',
                    'reflectance': {'type': 'rgb', 'value': [1.0, 1.0, 1.0]}
                }
            },
            'sensor': {
                'type': 'perspective',
                'fov': 60,
                'to_world': mi.ScalarTransform4f.look_at(
                    origin=[0, 0, -4],
                    target=[0, 0, 0],
                    up=[0, -1, 0]
                ),
                'film': {
                    'type': 'hdrfilm',
                    'width': res,
                    'height': res,
                    #'filter': { 'type': 'box' }
                }
            },
            'white': {
                'type': 'diffuse',
                'reflectance': {
                    'type': 'rgb',
                    'value': [0.885809, 0.698859, 0.666422],
                }
            },
            'sphere': {
                'type': 'sphere',
                'to_world': mi.ScalarTransform4f.scale(0.5),
            },
            'light': {
                'type': 'rectangle',
                'to_world': mi.ScalarTransform4f.translate([1.8, 0.5, 1.0]) @ mi.ScalarTransform4f.rotate([0, 1, 0], -130),
                # 'emitter': {
                #     'type': 'area',
                #     'radiance': {'type': 'rgb', 'value': [1.0, 0, 0]}
                # },
                'bsdf': {
                'type': 'ref',
                'id': 'white'
                },
                'emitter': {
                    'type': 'area',
                    'radiance': {
                        'type': 'rgb',
                        'value': [18.387, 13.9873, 6.75357],
                    }
                }
            },
            # 'light': {
            #     'type': 'constant',
            #     'radiance': 1.0
            # }
        }
scene_init = mi.load_dict(scene_dict)

In [3]:
ref_images = mi.render(scene_init)
mi.util.convert_to_bitmap(ref_images)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [32, 32],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 3 KiB of image data ]
]